In [17]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [18]:
def preprocess_images(folder, label, image_size=(224, 224)):
    data = []
    labels = []

    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, image_size)
        img = img / 255.0  # Normalize pixel values to the range [0, 1]

        data.append(img)
        labels.append(label)

    return np.array(data), np.array(labels)

In [19]:
dataset_pos = 'datasets/brain_mri_scan_images/positive'
dataset_neg = 'datasets/brain_mri_scan_images/negative'

positive_data, positive_labels = preprocess_images(dataset_pos, 1)
negative_data, negative_labels = preprocess_images(dataset_neg, 0)

x = np.concatenate((positive_data, negative_data))
y = np.concatenate((positive_labels, negative_labels))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# print(x_train, x_test)
# print(y_train, y_test)

print(f'Train: X={x_train.shape}, y={y_train.shape}')
print(f'Test: X={x_test.shape}, y={y_test.shape}')

input_shape = x_train.shape[1:4]
print(input_shape)


Train: X=(194, 224, 224, 1), y=(194,)
Test: X=(22, 224, 224, 1), y=(22,)
(224, 224, 1)


In [20]:
from tensorflow import keras
from keras import layers

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(2, 2), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(2, 2), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(1, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 223, 223, 32)      160       
                                                                 
 batch_normalization_1 (Batc  (None, 223, 223, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 110, 110, 64)      8256      
                                                                 
 batch_normalization_2 (Batc  (None, 110, 110, 64)     256       
 hNormalization)                                                 
                                                      

In [21]:
batch_size = 8
epochs = 15

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
22/22 [==============================] - 12s 72ms/step - loss: 5.4089 - accuracy: 0.5460 - val_loss: 0.6227 - val_accuracy: 0.6000
Epoch 2/15
22/22 [==============================] - 1s 61ms/step - loss: 4.5433 - accuracy: 0.5460 - val_loss: 0.7752 - val_accuracy: 0.6000
Epoch 3/15
22/22 [==============================] - 1s 63ms/step - loss: 3.0867 - accuracy: 0.5460 - val_loss: 0.5028 - val_accuracy: 0.6000
Epoch 4/15
22/22 [==============================] - 1s 65ms/step - loss: 0.2478 - accuracy: 0.5460 - val_loss: 0.4849 - val_accuracy: 0.6000
Epoch 5/15
14/22 [==================>...........] - ETA: 1:08 - loss: 0.2183 - accuracy: 0.4821

: 